In [17]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import functions as F

StatementMeta(, f3cbc6ab-cbec-4444-a5d2-3fa65eda61cf, 19, Finished, Available, Finished)

## Ejecutivo

In [13]:
df1 = spark.sql("SELECT * FROM lakehouseSilver.ejecutivo")
display(df1)

StatementMeta(, c19a4f8f-b25a-4d77-a211-79c2f325c435, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2f41528b-27eb-473f-a8d1-80767d51e49d)

In [14]:
df2 = spark.sql("SELECT * FROM lakehouseSilver.relacionclienteEjecutivo")
display(df2)

StatementMeta(, c19a4f8f-b25a-4d77-a211-79c2f325c435, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c8e802f1-1d9b-4703-abd4-cdca97814e1c)

In [15]:
dfEjecutivoBronze = df2.join(
    df1,
    on="idEjecutivo",
    how="inner"
).select("idCliente", "idEjecutivo", "Descripcion")

StatementMeta(, c19a4f8f-b25a-4d77-a211-79c2f325c435, 17, Finished, Available, Finished)

In [16]:
display(dfEjecutivoBronze)

StatementMeta(, c19a4f8f-b25a-4d77-a211-79c2f325c435, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a9892519-628f-4312-ba6c-2dea8035b242)

In [17]:
dfEjecutivoBronze.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("lakehouseSilver.dfEjecutivoBronze")

StatementMeta(, c19a4f8f-b25a-4d77-a211-79c2f325c435, 19, Finished, Available, Finished)

## Grupo

In [18]:
df3 = spark.sql("SELECT * FROM lakehouseSilver.grupoInteresEconomico")
display(df3)

StatementMeta(, c19a4f8f-b25a-4d77-a211-79c2f325c435, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f2c79187-94fd-416c-8e85-a770268173eb)

In [19]:
df4 = spark.sql("SELECT * FROM lakehouseSilver.relacionclienteGrupoInteresEconomico")
display(df4)

StatementMeta(, c19a4f8f-b25a-4d77-a211-79c2f325c435, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 374b44a7-25fd-4f3d-aec2-581b087134a6)

In [20]:
dfgrupoBronze = df4.join(
    df3,
    on = "idGrupo",
    how = "inner"
).select("idCliente", "idGrupo", "vcNombre")

StatementMeta(, c19a4f8f-b25a-4d77-a211-79c2f325c435, 22, Finished, Available, Finished)

In [21]:
display(dfgrupoBronze)

StatementMeta(, c19a4f8f-b25a-4d77-a211-79c2f325c435, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9891ea57-19c0-4ac3-8fba-2ecfb1f8a6f1)

In [22]:
dfgrupoBronze.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("lakehouseSilver.dfgrupoBronze")

StatementMeta(, c19a4f8f-b25a-4d77-a211-79c2f325c435, 24, Finished, Available, Finished)

## Activo

In [9]:
df5 = spark.sql("SELECT * FROM lakehouseSilver.project_activoDesc")
display(df5)

StatementMeta(, f3cbc6ab-cbec-4444-a5d2-3fa65eda61cf, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 63cfb755-89b0-4756-b65c-73c03d37a952)

In [2]:
df6 = spark.sql("SELECT * FROM lakehouseSilver.project_activoContrato")
display(df6)

StatementMeta(, f3cbc6ab-cbec-4444-a5d2-3fa65eda61cf, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 845c7c84-6d7c-4e12-9c15-39e8f0356111)

In [15]:
df_activoContrato = df6.alias("a")
df_activoDesc = df5.alias("b") 

StatementMeta(, f3cbc6ab-cbec-4444-a5d2-3fa65eda61cf, 17, Finished, Available, Finished)

In [20]:
dfactivoBronze = (
    df_activoContrato
    .join(df_activoDesc, on="idActivo", how="left")
    .select(
        F.col("a.idActivo").alias("idActivo"),
        F.col("a.numContrato").alias("numContrato"),
        F.col("b.codigocategoria").alias("codigocategoria")
    )
)

StatementMeta(, f3cbc6ab-cbec-4444-a5d2-3fa65eda61cf, 22, Finished, Available, Finished)

In [21]:
display(dfactivoBronze)

StatementMeta(, f3cbc6ab-cbec-4444-a5d2-3fa65eda61cf, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 122f2a95-dd73-484b-bbbc-d76c74ad65cd)

In [22]:
valores_categoria = [
    "001", "002", "003", "004", "999", "AV1", "AV2",
    "E01", "E02", "E03", "E04", "E05", "E06",
    "ED1", "ED2", "ED3", "P01", "P02", "P04",
    "RE", "V01", "V02", "V04", "V05", "V07",
    "V08", "V09", "V10", "V11", "V12", "V13",
    "V14", "V15", "V16", "V17", "V1R"
]

# Crear un array en Spark para seleccionar valores aleatorios
array_categoria = "array(" + ",".join([f"'{v}'" for v in valores_categoria]) + ")"

# Actualizar codigocategoria con un valor aleatorio cuando es NULL
dfactivoBronze = dfactivoBronze.withColumn(
    "codigocategoria",
    F.when(
        F.col("codigocategoria").isNull(),
        F.expr(f"element_at({array_categoria}, cast(rand() * {len(valores_categoria)} + 1 as int))")
    ).otherwise(F.col("codigocategoria"))
)

StatementMeta(, f3cbc6ab-cbec-4444-a5d2-3fa65eda61cf, 24, Finished, Available, Finished)

In [23]:
display(dfactivoBronze)

StatementMeta(, f3cbc6ab-cbec-4444-a5d2-3fa65eda61cf, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d28c4d2a-fd80-4d26-a8ea-eabf0fbef2e9)

In [24]:
dfactivoBronze.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("lakehouseSilver.dfactivoBronze")

StatementMeta(, f3cbc6ab-cbec-4444-a5d2-3fa65eda61cf, 26, Finished, Available, Finished)